### neo4j 取数据解析

In [5]:
from flask import Flask, jsonify, render_template
from py2neo import Graph

#app = Flask(__name__)
graph = Graph(
     host = "localhost", # neo4j 搭载服务器的ip地址，ifconfig可获取到
    http_port = 7474, # neo4j 服务器监听的端口号
    user = "neo4j", # 数据库user name，如果没有更改过，应该是neo4j
    password = "qiyu" # 自己设定的密码
)

def build_node(index,n):
    d = dict(zip(n['n'].keys(),n['n'].values()))
    d.update({"id":str(n['n'].identity)})
    d.update({"label":str(n['n'].labels)[1:]})
    return {"data": d}

def build_edge(r):
    d={
    "source": r['r'].start_node.identity,
    "target": r['r'].end_node.identity,
    "relationship":list(r['r'].relationships[0].types())[0]}
    return {"data":d}

In [7]:
#@app.route('/')
def index():
    return render_template('index.html')

#@app.route('/graph')
def get_graph():
    nodes = graph.run('MATCH (n) RETURN n').data()
    edges = graph.run('MATCH ()-[r]->() RETURN r').data()

    return jsonify(elements = {"nodes": [build_node(index,n) for index, n in enumerate(nodes)],
                                 "edges": [build_edge(r) for r in edges]})

In [14]:
nodes = graph.run('MATCH (n) RETURN n').data()
edges = graph.run('MATCH ()-[r]->() RETURN r').data()
data = {"nodes": [build_node(index,n) for index, n in enumerate(nodes)],
                                 "edges": [build_edge(r) for r in edges]}

data

{'edges': [{'data': {'relationship': 'EXTEND_TO', 'source': 0, 'target': 1}}],
 'nodes': [{'data': {'id': '0',
    'label': 'Model',
    'modelID': '1',
    'modelLink': 'https://baike.baidu.com/item/logistic%E5%9B%9E%E5%BD%92/2981575?fromtitle=%E9%80%BB%E8%BE%91%E5%9B%9E%E5%BD%92&fromid=17202449&fr=aladdin',
    'modelName': 'LR'}},
  {'data': {'id': '1',
    'label': 'Model',
    'modelID': '2',
    'modelLink': 'https://baike.baidu.com/item/LDA/13489644',
    'modelName': 'LDA'}}]}

In [17]:
node_indices = [e['data']['id'] for e in data['nodes']]
node_indices

['0', '1']

In [20]:
edge_starts =  [e['data']['source'] for e in data['edges']]
edge_end =  [e['data']['target'] for e in data['edges']]

In [33]:
from bokeh.plotting import figure
from bokeh.models import GraphRenderer,StaticLayoutProvider, Oval
from bokeh.palettes import Spectral8


In [41]:
graph = GraphRenderer()
graph.node_renderer.data_source.add(node_indices, 'index')
graph.node_renderer.data_source.add(Spectral8[:2], 'color')
graph.node_renderer.glyph = Oval(height=0.1, width=0.1)
graph.node_renderer.data_source.data = dict(
    index=node_indices,
    fill_color=Spectral8[:2])

graph.edge_renderer.data_source.data = dict(
    start=edge_starts,
    end=edge_end)

In [42]:
import math
circ = [int(i)*2*math.pi/8 for i in node_indices]
x = [math.cos(i) for i in circ]
y = [math.sin(i) for i in circ]

graph_layout = dict(zip(node_indices, zip(x, y)))
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)


plot = figure(title='Graph Layout Demonstration', x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools='', toolbar_location=None)
plot.renderers.append(graph)

output_file('graph.html')
show(plot)

In [37]:
circ,x,y

([0.0, 0.7853981633974483],
 [1.0, 0.7071067811865476],
 [0.0, 0.7071067811865475])

### 生成网页+链接

In [13]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.plotting import figure, output_file, show

output_file("openurl.html")

p = figure(plot_width=400, plot_height=400,
           tools="tap", title="Click the Dots")

source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    color=["navy", "orange", "olive", "firebrick", "gold"]
    ))

p.circle('x', 'y', color='color', size=20, source=source)

url = "http://www.colors.commutercreative.com/@color/"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(p)